In [ ]:
! pip install opencc
! pip install transformers

     |████████████████████████████████| 1.3MB 6.0MB/s 
ERROR: Operation cancelled by user
     |████████████████████████████████| 1.3MB 5.7MB/s 
     |████████████████████████████████| 890kB 19.5MB/s 
     |████████████████████████████████| 2.9MB 29.5MB/s 
     |████████████████████████████████| 1.1MB 46.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=f3acac19fb860e6a816f0e9e397ea045cadc1f667114317c188352530a066066
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!pip install gdown
!sudo apt-get install unzip
!gdown https://dl.fbaipublicfiles.com/MLQA/MLQA_V1.zip
!unzip MLQA_V1.zip
!rm MLQA_V1.zip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.
Downloading...
From: https://dl.fbaipublicfiles.com/MLQA/MLQA_V1.zip
To: /content/MLQA_V1.zip
100% 75.7M/75.7M [00:07<00:00, 9.55MB/s]
Archive:  MLQA_V1.zip
   creating: MLQA_V1/
   creating: MLQA_V1/dev/
  inflating: MLQA_V1/dev/dev-context-ar-question-ar.json  
  inflating: MLQA_V1/dev/dev-context-ar-question-de.json  
  inflating: MLQA_V1/dev/dev-context-ar-question-en.json  
  inflating: MLQA_V1/dev/dev-context-ar-question-es.json  
  inflating: MLQA_V1/dev/dev-context-ar-question-hi.json  
  inflating: MLQA_V1/dev/dev-context-ar-question-vi.json  
  inflating: MLQA_V1/dev/dev-context-ar-question-zh.json  
  inflating: MLQA_V1/dev/dev-context-de-question-ar.json  
  inflating: MLQA_V1/dev/dev-context-de-question-de.json  
  inflating: MLQA_V1/dev/dev-context-de-question-en

In [ ]:
# MUSE
import opencc
converter = opencc.OpenCC('t2s.json')
print(converter.convert('漢字'))  # 漢字

convert_dict={}
with open('en-zh.txt', 'r') as f:
    for line in f:
        en, zh = line.strip().split()
        if en not in convert_dict:
            convert_dict[en] = converter.convert(zh)
# print(len(convert_dict))
            
import json
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import nltk
from transformers import MBartForConditionalGeneration, MBartTokenizer
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-en-ro")
tokenizer = MBartTokenizer.from_pretrained("facebook/mbart-large-en-ro")
article = "UN Chief Says There Is No Military Solution in Syria"
batch = tokenizer.prepare_seq2seq_batch(src_texts=[article], src_lang="en_XX")
translated_tokens = model.generate(**batch, decoder_start_token_id=tokenizer.lang_code_to_id["ro_RO"])
translation = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words('english')) 

def convert_a_line(s):
    ret = ''
    for word in word_tokenize(s.strip()):
        t = word.lower()
        if t in stop_words:
            continue
#         ret += t
        st = stemmer.stem(t)
        if t in convert_dict:
            ret += convert_dict[word.lower()]
        elif st in convert_dict:
            ret += convert_dict[st]
        elif len(st) > 5 and st + 'e' in convert_dict:
            ret += convert_dict[st + 'e']
        else:
#             ret += word
            ret += '[UNK]'
#             ret += ''
#             print(word)
    return ret

        

def read_MLQA_to_chinese(path):
    with open(path, 'r') as f:
        MLQA_dict = json.load(f)
    contexts = []
    questions = []
    answers = []

    for group in MLQA_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    before_answer = context[:answer['answer_start']]
                    after_answer = context[answer['answer_start']:]
                    c_before_answer = convert_a_line(before_answer)
                    c_after_answer = convert_a_line(after_answer)
                    c = c_before_answer + c_after_answer
                    q = convert_a_line(question)
                    a = {'text': convert_a_line(answer['text']), 'answer_start': len(c_before_answer)}
                    if '从业人员利用' in c:
                        print('****')
                        print(c)
                        print(q)
                        print(a)
                    contexts.append(c)
                    questions.append(q)
                    answers.append(a)

    return contexts, questions, answers

train_contexts_muse, train_questions_muse, train_answers_muse = read_MLQA_to_chinese('MLQA_V1/test/test-context-en-question-en.json')
print(train_contexts_muse[12])
print(train_questions_muse[12])
print(train_answers_muse[12])
print(len(train_contexts_muse))

汉字


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
****
从业人员利用[UNK]温暖区域子午线点数意图刺激环流点数诱导[UNK]流血液[UNK][UNK][UNK]对待条件[UNK][UNK]冷冷[UNK][UNK]杨缺陷[UNK]中文药品[UNK]认领[UNK][UNK]冷冷潮湿尸体[UNK]服务转[UNK][UNK]索赔[UNK]尤其有效治疗慢性问题[UNK][UNK][UNK]条件[UNK][UNK]弱[UNK][UNK][UNK][UNK]扁que[UNK][UNK][UNK][UNK][UNK]bce[UNK][UNK]一著名[UNK]医师中文古物首先专家[UNK][UNK]讨论福利[UNK]针灸经典工作扁que[UNK][UNK]断言[UNK][UNK]加上新能量尸体[UNK]对待过剩[UNK]条件[UNK]
使用温暖区域[UNK]
{'text': '[UNK]', 'answer_start': 6}
原文矩形基地[UNK][UNK]万里[UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK]新郎箱[UNK][UNK]矩形面积测量[UNK][UNK]万里[UNK][UNK][UNK][UNK][UNK][UNK]受限领空[UNK]面积连线内部[UNK]测试网站[UNK]nts[UNK]公路网路[UNK][UNK]公路领先南水星西边[UNK]平面[UNK]领先东北湖泊[UNK]广[UNK]新郎湖泊公路跑传球[UNK]丘陵[UNK]公路[UNK]led地雷新郎盆地[UNK]改进sinc[UNK][UN

In [ ]:
import json
def read_MLQA(path):
    with open(path, 'r') as f:
        MLQA_dict = json.load(f)
    contexts = []
    questions = []
    answers = []

    for group in MLQA_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

val_contexts, val_questions, val_answers = read_MLQA('MLQA_V1/dev/dev-context-en-question-en.json')
zh_contexts, zh_questions, zh_answers = read_MLQA('MLQA_V1/dev/dev-context-zh-question-zh.json')
train_contexts, train_questions, train_answers = read_MLQA('MLQA_V1/test/test-context-en-question-en.json')


print(len(train_contexts))
print(len(val_contexts))
print(len(zh_contexts))
print(val_questions[0], val_answers[0])

11590
1148
504
Does an infection for Sandflies go away over time? {'text': 'remains infected for its lifetime', 'answer_start': 571}


In [ ]:
use_original_p = 0.8 # = 1-use_muse
use_original = int(use_original_p * len(train_contexts))

train_contexts[use_original:] = train_contexts_muse[use_original:]
train_questions[use_original:] = train_questions_muse[use_original:]
train_answers[use_original:] = train_answers_muse[use_original:]

In [ ]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx].lower() == gold_text.lower():
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters
        else:
#             print(context[start_idx:end_idx])
#             print(gold_text)
#             print(context[start_idx:end_idx] == gold_text)
#             print(context)
#             print(start_idx)
            answer['answer_end'] = end_idx
            print('***')
#             jkalsdjkl

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)
add_end_idx(zh_answers, zh_contexts)
print(val_questions[0], val_answers[0])
print(zh_questions[2], zh_answers[2])

***
***
***
***
***
***
***
***
***
***
***
***
Does an infection for Sandflies go away over time? {'text': 'remains infected for its lifetime', 'answer_start': 571, 'answer_end': 604}
俄罗斯有多少队获得参赛资格？ {'text': '十支', 'answer_start': 153, 'answer_end': 155}


In [ ]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)
zh_encodings = tokenizer(zh_contexts, zh_questions, truncation=True, padding=True)

In [ ]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
#         print(answers[i]['answer_end'] - 1)
        end_positions.append(encodings.char_to_token(i, max(answers[i]['answer_end'] - 1, 0)))
        # if None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)
add_token_positions(zh_encodings, zh_answers)

In [ ]:
import torch

class MLQADataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = MLQADataset(train_encodings)
val_dataset = MLQADataset(val_encodings)
zh_dataset = MLQADataset(zh_encodings)

In [ ]:
def compute_f1(predicted, true):
    c = len(set(predicted) & set(true))
    l1 = len(predicted)
    l2 = len(true)
    if(l1 + l2 == 0):
        return 1
    f1 = 2*c/(l1+l2)
    return f1
    
def compute_em(predicted, true):
    return int(predicted == true)

In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW
from transformers import BertForQuestionAnswering

# input_ids = None
# attention_mask = None
# start_positions = None
# end_positions = None
# model = None
# torch.cuda.empty_cache()

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

model = BertForQuestionAnswering.from_pretrained('bert-base-multilingual-cased')
model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)
zh_loader = DataLoader(zh_dataset, batch_size=8, shuffle=True)
print(len(train_loader))
print(len(val_loader))
learning_rate = 5e-6
dummy_index = 0
collect = []
optim = AdamW(model.parameters(), lr=learning_rate)


val_batch = 300
max_epoch = 3
train_batch = len(train_loader)
for epoch in range(max_epoch):
    for batch_idx, batch in enumerate(train_loader):
        model.train()
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        optim.step()
        
        if batch_idx % val_batch == 0 or batch_idx == train_batch - 1:
            print("Epoch: {}/{}, batch: {}/{}, {:%}".format(epoch, max_epoch, batch_idx, train_batch, batch_idx/train_batch))
            dummy_index = epoch * train_batch + batch_idx
            model.eval()
            eval_cnt = 0
            F1 = 0.0
            EM = 0.0
            for batch_idx, batch in enumerate(val_loader):
                optim.zero_grad()
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                start_positions = batch['start_positions'].to(device)
                end_positions = batch['end_positions'].to(device)
                outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
                samples_in_batch = len(input_ids)
                for i in range(samples_in_batch):
                    predict_start = int(outputs[1][i].argmax().cpu())
                    predict_end = int(outputs[2][i].argmax().cpu())
                    true_start = int(start_positions[i].cpu())
                    true_end = int(end_positions[i].cpu())
                    F1 += compute_f1(range(predict_start, predict_end), range(true_start, true_end))
                    EM += compute_em(range(predict_start, predict_end), range(true_start, true_end))
                eval_cnt += samples_in_batch
            F1 /= eval_cnt
            EM /= eval_cnt
            print("English eval score: F1:{}, EM:{}".format(F1, EM))
            eng_f1, eng_em = F1, EM

            eval_cnt = 0
            F1 = 0.0
            EM = 0.0
            for batch_idx, batch in enumerate(zh_loader):
                optim.zero_grad()
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                start_positions = batch['start_positions'].to(device)
                end_positions = batch['end_positions'].to(device)
                outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
                samples_in_batch = len(input_ids)
                for i in range(samples_in_batch):
                    predict_start = int(outputs[1][i].argmax().cpu())
                    predict_end = int(outputs[2][i].argmax().cpu())
                    true_start = int(start_positions[i].cpu())
                    true_end = int(end_positions[i].cpu())
                    F1 += compute_f1(range(predict_start, predict_end), range(true_start, true_end))
                    EM += compute_em(range(predict_start, predict_end), range(true_start, true_end))
                eval_cnt += samples_in_batch
            F1 /= eval_cnt
            EM /= eval_cnt
            print("Chinese eval score: F1:{}, EM:{}".format(F1, EM))
            print('collect data: ', eng_f1, eng_em, F1, EM, dummy_index, epoch, learning_rate)
            collect.append((eng_f1, eng_em, F1, EM, dummy_index, epoch, learning_rate))


cuda


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-bas

1449
144
Epoch: 0/3, batch: 0/1449, 0.000000%
English eval score: F1:0.1906445901621307, EM:0.18118466898954705
Chinese eval score: F1:0.07257237134085756, EM:0.051587301587301584
collect data:  0.1906445901621307 0.18118466898954705 0.07257237134085756 0.051587301587301584 0 0 5e-06
Epoch: 0/3, batch: 300/1449, 20.703934%
English eval score: F1:0.2942128158851099, EM:0.22038327526132404
Chinese eval score: F1:0.20196747593864367, EM:0.1130952380952381
collect data:  0.2942128158851099 0.22038327526132404 0.20196747593864367 0.1130952380952381 300 0 5e-06
Epoch: 0/3, batch: 600/1449, 41.407867%
English eval score: F1:0.44795617763824164, EM:0.3423344947735192
Chinese eval score: F1:0.29705462129441734, EM:0.1884920634920635
collect data:  0.44795617763824164 0.3423344947735192 0.29705462129441734 0.1884920634920635 600 0 5e-06
Epoch: 0/3, batch: 900/1449, 62.111801%
English eval score: F1:0.5146267417662432, EM:0.4076655052264808
Chinese eval score: F1:0.34943915302615247, EM:0.2321428

In [ ]:
with open('MUSE_lr_' + str(learning_rate) + '.csv', 'w') as f:
    f.write("En_F1,En_EM,Zh_F1,Zh_EM,batch_size,epoch,lr\n")
    for line in collect:
        sline = list(map(lambda x: str(x), line))
        sline = ','.join(sline)
        f.write(sline+"\n")